# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)

X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [3]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df.isnull().sum()

id                        0
message                   0
original                  0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state = 42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
Y_pred = pipeline.predict(X_test)

In [11]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.62      0.60      0.61      1088
          1       0.80      0.82      0.81      2231
          2       0.80      0.21      0.33        38

avg / total       0.74      0.74      0.74      3357

request
             precision    recall  f1-score   support

          0       0.81      0.92      0.86      2142
          1       0.82      0.61      0.70      1215

avg / total       0.81      0.81      0.80      3357

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3354
          1       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      3357

aid_related
             precision    recall  f1-score   support

          0       0.77      0.91      0.83      2043
          1       0.80      0.59      0.68      1314

avg / total       0.78      0.78      0.77      3357

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 10)

In [16]:
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.285336856010568, total=   7.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.2s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.3051519154557464, total=   7.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   20.2s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.2919418758256275, total=   7.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.2884191985909291, total=   7.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.3130779392338177, total=   6.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.3099955966534566, total=   7.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.30647291941875826, total=   6.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.30735358872743285, total=  15.3s
[CV] clf__estimator__min_samples_split=2, clf

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 15.2min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
Y_pred_tuned = tuned_model.predict(X_test)

In [19]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred_tuned[:, i]))

related
             precision    recall  f1-score   support

          0       0.70      0.57      0.63      1088
          1       0.80      0.88      0.84      2231
          2       0.56      0.39      0.46        38

avg / total       0.77      0.78      0.77      3357

request
             precision    recall  f1-score   support

          0       0.84      0.91      0.87      2142
          1       0.82      0.69      0.75      1215

avg / total       0.83      0.83      0.83      3357

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3354
          1       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      3357

aid_related
             precision    recall  f1-score   support

          0       0.82      0.88      0.85      2043
          1       0.79      0.69      0.74      1314

avg / total       0.81      0.81      0.81      3357

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [39]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__max_depth': [7], 
              'clf__estimator__min_samples_split': [10],
              'clf__estimator__min_samples_leaf': [10]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, verbose = 10)

In [40]:
tuned_model2 = cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5, score=0.36459709379128136, total=   5.0s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5, score=0.35623073535887273, total=   4.9s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   13.7s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=True, vect__min_df=5, score=0.3703214442976662, total=   4.9s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   26.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5, score=0.35667107001321, total=   4.5s
[CV] clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   33.2s remaining:    0.0s


[CV]  clf__estimator__max_depth=7, clf__estimator__min_samples_leaf=10, clf__estimator__min_samples_split=10, tfidf__use_idf=False, vect__min_df=5, score=0.36900044033465434, total=   4.5s


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   39.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   39.6s finished


In [41]:
Y_pred_tuned2 = tuned_model2.predict(X_test)

In [42]:
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred_tuned2[:, i]))

related
             precision    recall  f1-score   support

          0       0.53      0.84      0.65      1088
          1       0.89      0.65      0.75      2231
          2       0.00      0.00      0.00        38

avg / total       0.76      0.71      0.71      3357

request
             precision    recall  f1-score   support

          0       0.81      0.90      0.85      2142
          1       0.78      0.63      0.69      1215

avg / total       0.80      0.80      0.79      3357

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3354
          1       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      3357

aid_related
             precision    recall  f1-score   support

          0       0.78      0.88      0.83      2043
          1       0.77      0.61      0.68      1314

avg / total       0.77      0.78      0.77      3357

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [43]:
pickle.dump(tuned_model, open('ml_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.